[View in Colaboratory](https://colab.research.google.com/github/Dark-Sied/Aidapp/blob/master/Hinglish_wordembedings.ipynb)

In [1]:
!pip install gensim

    100% |████████████████████████████████| 23.5MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 11.9MB/s 
    100% |████████████████████████████████| 133kB 24.3MB/s 
    100% |████████████████████████████████| 4.6MB 5.5MB/s 
    100% |████████████████████████████████| 61kB 16.3MB/s 
    100% |████████████████████████████████| 552kB 20.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
import bs4 as bs
import nltk
import re
from keras.preprocessing.text import one_hot
from nltk.corpus import stopwords
from gensim.models import Word2Vec

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding



Using TensorFlow backend.


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stop_words = set(stopwords.words('english'))

In [8]:
!ls

english_text.csv  hinglish_new.w2v   hinglish_text.csv	stopwords_list.txt
harry.w2v	  HinglishSentiment  sample_data	word_embeddings_hp


In [9]:
stop_word = []
with open("stopwords_list.txt") as f:
  word = f.readlines()
#   print(word)
  for i in word:
#     print(i)
    stop_word.append(i.rstrip())
  print(stop_word)

['ke', 'ka', 'ek', 'mein', 'ki', 'hai', 'yah', 'aur', 'se', 'hain', 'ko', 'par', 'iss', 'hota', 'jo', 'kar', 'me', 'gaya', 'karne', 'kiya', 'liye', 'apne', 'ne', 'bani', 'nahi', 'toh', 'hi', 'ya', 'avam', 'diya', 'ho', 'iska', 'tha', 'dhvara', 'hua', 'tak', 'saath', 'karna', 'vaale', 'baad', 'liya', 'aap', 'kuchh', 'sakte', 'kisi', 'ye', 'iska', 'sabse', 'ismein', 'the', 'do', 'hone', 'vah', 've', 'karte', 'bahut', 'kaha', 'varg', 'kai', 'karein', 'hoti', 'apni', 'unke', 'thi', 'yadi', 'hui', 'jaa', 'na', 'ise', 'kehte', 'kahte', 'jab', 'hote', 'koi', 'hue', 'va', 'abhi', 'jaise', 'sabhi', 'karta', 'unki', 'tarah', 'uss', 'aadi', 'kul', 'raha', 'iski', 'sakta', 'rahe', 'unka', 'issi', 'rakhein', 'apna', 'pe', 'uske']


In [3]:
!ls

english_text.csv  HinglishSentiment  hinglish.w2v  word_embeddings_hp
harry.w2v	  hinglish_text.csv  sample_data


In [10]:
hinglish = pd.read_csv("hinglish_text.csv")
hinglish.head()

,Unnamed: 0,text
0,0,ek din mujhe yunivarsal kampani ke karl laeeml...
1,1,jess robins nam ke ek yuva jo aisene kampani k...
2,2,us rat mainne robins ko dinar par aamantrit ki...
3,3,main sataven aasaman par tha prastav itana sha...
4,4,halanki mere shak ne sir uthana shuroo kar diy...


In [0]:
hinglish["labels"] = 1

In [12]:
hinglish.head()

,Unnamed: 0,text,labels
0,0,ek din mujhe yunivarsal kampani ke karl laeeml...,1
1,1,jess robins nam ke ek yuva jo aisene kampani k...,1
2,2,us rat mainne robins ko dinar par aamantrit ki...,1
3,3,main sataven aasaman par tha prastav itana sha...,1
4,4,halanki mere shak ne sir uthana shuroo kar diy...,1


In [13]:
english = pd.read_csv("english_text.csv")
english.head()

,Unnamed: 0,text
0,0,Harry Potter and the Sorcerers Stone
1,1,CHAPTER ONE
2,2,THE BOY WHO LIVED
3,3,Mr and Mrs Dursley of number four Privet Drive...
4,4,Mr Dursley was the director of a firm called G...


In [14]:
english["labels"] = 0
english.head()

,Unnamed: 0,text,labels
0,0,Harry Potter and the Sorcerers Stone,0
1,1,CHAPTER ONE,0
2,2,THE BOY WHO LIVED,0
3,3,Mr and Mrs Dursley of number four Privet Drive...,0
4,4,Mr Dursley was the director of a firm called G...,0


In [15]:
final_data = hinglish.append(english)
# final_data.sample(frac = 1).reset_index(drop = True)
final_data.head()

,Unnamed: 0,text,labels
0,0,ek din mujhe yunivarsal kampani ke karl laeeml...,1
1,1,jess robins nam ke ek yuva jo aisene kampani k...,1
2,2,us rat mainne robins ko dinar par aamantrit ki...,1
3,3,main sataven aasaman par tha prastav itana sha...,1
4,4,halanki mere shak ne sir uthana shuroo kar diy...,1


In [16]:
from sklearn.utils import shuffle
final_data = shuffle(final_data)
final_data

,Unnamed: 0,text,labels
7292,7292,Angelina scoring,0
26114,26114,arent we still underground,0
27355,27355,Um said Cho Well just thought Id say hello bye...,0
33915,33915,lined with more portraits of famous Healers an...,0
2142,2142,Har Ladki Ke Liye Ek Account Aajkal ek ladki ...,1
10663,10663,when he hit the ground the pipe leveled out an...,0
31194,31194,Right said Harry vaguely putting down the bott...,0
2194,2194,Hagrids chest swelled at these last words Harr...,0
10142,10142,Gryffindor common room where the fire had burn...,0
52014,52014,From above came Bellatrixs voice,0


In [0]:
    hing_words = [[w for w in s.lower().split() if w not in stop_word]for s in hinglish]
    eng_words = [[w for w in s.lower().split() if w not in stop_words]for s in english]



In [67]:
!ls

english_text.csv  HinglishSentiment  hinglish.w2v  stopwords_list.txt
harry.w2v	  hinglish_text.csv  sample_data   word_embeddings_hp


In [0]:
hing_model = Word2Vec.load("hinglish.w2v")
eng_model = Word2Vec.load("harry.w2v")

In [0]:
vocab = dict()

hing_model.wv["aa"]
for i in hing_model.wv.vocab:
  vocab[i] = hing_model.wv[i]
#   print(i)

for i in eng_model.wv.vocab:
  vocab[i] = eng_model.wv[i]
  
  


In [24]:
len(vocab.keys())

8009

In [39]:
# final_data["text"]

49751    After another quarter of an hour however he wa...
10360    enemy and the Basilisk flees only from the cro...
49249    That is a valuable object said Scrimgeour watc...
40132    Okay he said but did not move He had just hear...
1376     Wood was now looking as though all his dreams ...
34011                                               happen
39495                      always did when she was furious
1751     Keep out of the way until you catch sight of t...
9849     contented himself with scrawling a note to Ron...
35312    subject of Bellatrix and her fellow torturers ...
44970    The frosty grass crunched underfoot as they st...
8344             getting all the credit for his dirty work
21178    Harry could feel his face burning Snape was pa...
49855    And one day a year after he joined Master Regu...
1431     Dayaluta shaktishaali hai khaaskar parivaar ko...
5920     somewhat higher than usual I think youre being...
30133    These things often skip  er  three generations.

In [25]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(vocab.keys())
vocab_size = len(t.word_index) + 1
print(vocab_size)
# integer encode the documents
encoded_docs = t.texts_to_sequences(final_data["text"])
# print(encoded_docs)

8010


In [0]:
# vocab_size = len(t.word_index)+1
vocab_ls = list(vocab)
# print(vocab)
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 150))
for i,word in enumerate(vocab_ls):
#   print(vocab[word])
  embedding_vector = vocab[word]
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

# print(model.wv['stuff'])

In [27]:
#padding
max_length = 10000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
print(padded_docs.shape)

[[7653    0    0 ...    0    0    0]
 [7073 6101    0 ...    0    0    0]
 [1266 7927 6181 ...    0    0    0]
 ...
 [3158 2723 2119 ...    0    0    0]
 [6181 6181 6181 ...    0    0    0]
 [ 771 5888 7328 ...    0    0    0]]
(59176, 10000)


In [0]:
labels = final_data['labels']

In [29]:
clf = Sequential()
clf.add(Embedding(vocab_size,150 , input_length= max_length,  weights=[embedding_matrix], trainable = False))
clf.add(Flatten())
clf.add(Dense(1, activation="sigmoid"))

clf.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])
print(clf.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10000, 150)        1201500   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1500000)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1500001   
Total params: 2,701,501
Trainable params: 1,500,001
Non-trainable params: 1,201,500
_________________________________________________________________
None


In [30]:
clf.fit(padded_docs, labels, epochs = 2, batch_size = 32)

Epoch 1/2
59176/59176 [==============================] - 423s 7ms/step - loss: 1.2168 - acc: 0.9245
Epoch 2/2
59176/59176 [==============================] - 422s 7ms/step - loss: 1.2175 - acc: 0.9245


In [46]:
s = ["chal kahin door jate hai"]
encode = t.texts_to_sequences(s)
padded_docs = pad_sequences(encode, maxlen=max_length, padding='post')


clf.predict_classes(padded_docs)

array([[0]], dtype=int32)

In [47]:
padded_docs
print(padded_docs[0])

[ 460  900 1153 ...    0    0    0]


In [38]:

t.word_index["chal"]
t.word_index["kahin"]
t.word_index["chate"]


KeyError: ignored